In [1]:
suppressWarnings(suppressMessages({
    library(rvest)
    library(readr)
    library(dplyr)
    library(ggplot2)
    library(reshape2)
    library(tidyverse)
    }))

# Data scraping and data cleaning

# Step 0: Create a vector with all urls of interest

# Step 1: Define a function that scrapes a given wikipedia article (one for prem, one for football league)
# and collects the correct table, with the correct data (variables/columns)

# Step 2: Define a function that cleans a given table from wikipedia (i.e. removes [a],[b], (C) (R) etc)

# Step 3: Use the function from step 2 on all dataframes from step 1 and then do a full_join(). Check so
# that all data looks okay

# Step 4: Do a heatmap. Required data is: team name, position and year. Should be long format, i.e. to each 
# (team,year) there should be a corresponding position.

# Step 5: Do a lineplot of the points of the team in last place, 12th, 6th and 1th place and study if there has been
# a change with time. In the report, start off with a comparison of two years, compare distributions of points
# the entire thing! What is the wanted format?



In [2]:
# Step 0: Getting all URLs for 

# The parts that all URLs share
baseurl1 <- "https://en.wikipedia.org/wiki/"
baseurl2 <- "%E2%80%93"
FL <- "_Football_League"


# Years the FL took place (years used in URLs)
years1FL <- as.character(seq(1950,1999))
years2FL <- as.character(seq(51,92))


# Creating a vector with all URLs for the Football League (period 1951 to 1992)
FL_urls <- c() 
for (i in 1:42){
    next_url <- paste0(baseurl1,years1FL[i],baseurl2,years2FL[i],FL)
    FL_urls <- c(FL_urls, next_url)
    }

In [3]:
# Step 1:


# Premier league
table_collect_prem <- function(url){
    page <- read_html(url)
    
    tables <- page %>%
      html_nodes("table.wikitable") %>% 
      html_table(fill = TRUE)  
    
    return(tables[[4]]) # Gives the 4th table
    }


# Football league
table_collect_foot <- function(url){
    page <- read_html(url)
    
    tables <- page %>%
      html_nodes("table.wikitable") %>% 
      html_table(fill = TRUE)  
    
    return(tables[[1]]) # Gives the 4th table
    }

# Saves relevant data 
# FL 51 to 92.

var_extract_pos <- function(table,year){
    table <- table[,c(1,2)]
    table <- table %>%
        rename(Position = 1, Team = 2)
    return(table)
}



var_extract_pts <- function(table,year){
    table <- table[, c(names(table)[2], "Pts")]
    table <- table %>%
        rename(!!as.character(year) := Pts)
    table %>%
        rename(Team = 1)
    return(table)
}



# Step 2:
# Remove annoying crap ( (C), (R) [a] etc...)

remove_crap <- function(table){
    table[] <- lapply(table, function(x) gsub("\\[a\\]|\\[b\\]|\\[c\\]\\[d\\]$", "", x))
    table[] <- lapply(table, function(x) gsub(" \\(R\\)| \\(C\\)| \\(O\\)$", "", x)) 
    table[] <- lapply(table, function(x) gsub("\\[a\\]|\\[b\\]|\\[c\\]\\[d\\]$", "", x))
    table[] <- lapply(table, function(x) gsub("\\(R\\)|\\(C\\)|\\(O\\)$", "", x))
    table[] <- lapply(table, function(x) gsub("\\[c\\]$", "", x))
    return(table)
}

In [7]:
# Step 3: Joining all the data (position)

all_scraping_pos <- function(url,year){
    output <- table_collect_foot(url) %>%
        var_extract_pos(year) %>%
        remove_crap()
    return(output)
    }

df_full <- all_scraping_pos(FL_urls[1],years1FL[2])
for (i in 2:42){
    next_df <- all_scraping_pos(FL_urls[i],years1FL[i+1])
    df_full <- full_join(df_full,next_df,by="Team")
    }

In [5]:
# Step 3: Joining all the data (points)

all_scraping_pts <- function(url,year){
    output <- table_collect_foot(url) %>%
        var_extract_pts(year) %>%
        remove_crap()
    return(output)
    }

df_full <- all_scraping_pts(FL_urls[1],years1FL[2])
for (i in 2:42){
    next_df <- all_scraping_pts(FL_urls[i],years1FL[i+1])
    df_full <- full_join(df_full,next_df,by="Team")
    }

ERROR: [1m[33mError[39m in `full_join()`:[22m
[1m[22m[33m![39m Join columns in `y` must be present in the data.
[31m✖[39m Problem with `Team`.


In [ ]:
# Teams not in the FL are all given joint position 23
df_fulltest <- df_full
df_full[is.na(df_full)] <- "23" 

In [ ]:
df_full_long <- pivot_longer(
    df_full,
    cols = starts_with("Pos"),
    names_to = "Year",
    values_to = "Position"
)

df_full_longtest <- pivot_longer(
    df_fulltest,
    cols = starts_with("Pos"),
    names_to = "Year",
    values_to = "Position"
)


In [ ]:
df_full_long$Position <- as.numeric(df_full_long$Position)
df_full_longtest$Position <- as.numeric(df_full_longtest$Position)

In [ ]:
ggplot(df_full_long, aes(x = Year, y = Team, fill = Position)) +
  geom_tile() +
  labs(title = "Correlation Heatmap",
       x = "Football club",
       y = "Year")+
scale_fill_gradient(
    low = "blue",      # Color for low values
    high = "red")+  
theme(axis.text.y = element_text(angle = 45, hjust = 1)) 

ggplot(df_full_longtest, aes(x = Year, y = Team, fill = Position)) +
  geom_tile() +
  labs(title = "Correlation Heatmap",
       x = "Football club",
       y = "Year")+
scale_fill_gradient(
    low = "blue",      # Color for low values
    high = "red")+  
theme(axis.text.y = element_text(angle = 45, hjust = 1)) 

In [ ]:
url <- "https://en.wikipedia.org/wiki/1987%E2%80%9388_Football_League"

table_collect_foot(url) %>%
    var_extract_pts(1987)